In [1]:
import pandas as pd
import geopandas as gpd
import os
from osgeo import ogr
import numpy as np

In [2]:
save_output_file = False
output_path = 'Results_REMIND_JRC.csv'
base_folder = 'JRC-IDEES-2021'

#### Global treatment of JRC data and REMIND 

##### Steps :
- Use the excel sheets from JRC for each country
- Create a dataframe for JRC based on values of 2020 for the transport categories
- Add the shares of each country in the global EU27 values t the dataframe
- Distribute EU27 agregated value of REMIND over all categories
- Save new final output file with distributed data (all caegories and every year)

In [3]:
transport_data = pd.read_csv("REMIND_Transport_Output.mif", sep=';')

In [4]:
transport_data = transport_data.drop(columns={"Model", "Scenario","2055", "2060", "2070", "2080", "2090", "2100", "2110", "2130", "2150"})

In [5]:
transport_data.Region.unique()
Variable = transport_data['Variable'].str.split('|', expand=True)

In [6]:
transport_EU = transport_data.query("Region == 'EU27'")

In [7]:
def split_variable(df):
    variables = df['Variable'].str.split('|', expand=True)
    num_cols =  variables.shape[1]
    new_column_names = [f'Variable_{i}' for i in range(num_cols)]
    df[new_column_names] = variables
    return(df)

In [8]:
transport_EU = split_variable(transport_EU)

C:\Users\mar.eco\AppData\Local\Temp\ipykernel_20948\3017919102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_names] = variables
C:\Users\mar.eco\AppData\Local\Temp\ipykernel_20948\3017919102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column_names] = variables
C:\Users\mar.eco\AppData\Local\Temp\ipykernel_20948\3017919102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [9]:
final_energy = transport_EU[(transport_EU["Variable_0"] == "FE") & transport_EU["Variable_1"].isin(["Transport", "Bunkers"])]

In [10]:
final_energy

,Region,Variable,Unit,2005,2010,2015,2020,2025,2030,2035,...,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,Variable_11,Variable_12
82801,EU27,FE|Transport|Bunkers|Freight|International Shi...,EJ/yr,0.009081,0.016569,0.026675,0.086255,1.752095e-01,1.756969e-01,0.182370,...,Freight,International Shipping,Liquids,Biomass,None,None,None,None,None,None
82826,EU27,FE|Transport|Bunkers|Freight|International Shi...,EJ/yr,1.928230,1.902167,1.887295,1.828646,1.735520e+00,1.720969e+00,1.700257,...,Freight,International Shipping,Liquids,Fossil,None,None,None,None,None,None
82851,EU27,FE|Transport|Bunkers|Freight|International Shi...,EJ/yr,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,Freight,International Shipping,Liquids,Hydrogen,None,None,None,None,None,None
82876,EU27,FE|Transport|Bunkers|Pass|International Aviati...,EJ/yr,0.013064,0.021189,0.028266,0.066779,1.134717e-01,1.123739e-01,0.114467,...,Pass,International Aviation,Liquids,Biomass,None,None,None,None,None,None
82901,EU27,FE|Transport|Bunkers|Pass|International Aviati...,EJ/yr,1.248860,1.286443,1.216144,1.140997,1.073285e+00,1.046982e+00,1.011112,...,Pass,International Aviation,Liquids,Fossil,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135672,EU27,FE|Transport|Pass|Aviation|Hydrogen,EJ/yr,0.000000,0.000000,0.000000,0.000000,1.968411e-08,4.825360e-08,0.000001,...,Aviation,Hydrogen,None,None,None,None,None,None,None,None
135675,EU27,FE|Transport|Pass|Aviation|Liquids,EJ/yr,1.535866,1.576815,1.370199,1.343418,1.328005e+00,1.338677e+00,1.326241,...,Aviation,Liquids,None,None,None,None,None,None,None,None
135799,EU27,FE|Transport|Pass|Aviation|Liquids||Fossil,EJ/yr,1.520136,1.551127,1.338982,1.267384,1.201112e+00,1.206651e+00,1.183565,...,Aviation,Liquids,,Fossil,None,None,None,None,None,None
135802,EU27,FE|Transport|Pass|Aviation|Liquids||Hydrogen,EJ/yr,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,Aviation,Liquids,,Hydrogen,None,None,None,None,None,None


In [11]:
def filter_exact_path(df, path):
    variables_split = df['Variable'].str.split('|', expand=True)
    expected_depth = len(path.split('|'))
    
    # Replace None/NaN with empty strings to avoid errors
    cleaned = variables_split.iloc[:, :expected_depth].fillna('')
    
    joined_path = cleaned.agg('|'.join, axis=1)
    actual_depth = variables_split.notna().sum(axis=1)

    mask = (joined_path == path) & (actual_depth == expected_depth)
    return df[mask]

In [12]:
def filter_category_with_subpaths(df, category_prefix):
    return df[df["Variable"].str.startswith(category_prefix)]

In [13]:
categories_REMIND = [
    "FE|Transport|Freight|Road|Heavy",
    "FE|Transport|Freight|Road|Light",
    "FE|Transport|Pass|Road|Bus",
    "FE|Transport|Pass|Road|LDV|Four Wheelers",
    "FE|Transport|Pass|Road|LDV|Two Wheelers",
    "FE|Transport|Bunkers|Freight|International Shipping",
    "FE|Transport|Freight|Domestic Shipping",
    "FE|Transport|Pass|Domestic Aviation",
    "FE|Bunkers|Pass|International Aviation",
    "FE|Transport|Pass|Aviation",
    "FE|Transport|Pass|Rail",
    "FE|Transport|Freight|Rail"
]

In [190]:
'''filtered_data = {}
for category in categories_REMIND:
    filtered_data[category] = filter_exact_path(final_energy, category)'''

'filtered_data = {}\nfor category in categories_REMIND:\n    filtered_data[category] = filter_exact_path(final_energy, category)'

In [14]:
filtered_data = {}
for category in categories_REMIND:
    filtered_data[category] = filter_category_with_subpaths(final_energy, category)

In [31]:
filtered_data["FE|Transport|Pass|Aviation"]

,Region,Variable,Unit,2005,2010,2015,2020,2025,2030,2035,...,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,Variable_11,Variable_12
135300,EU27,FE|Transport|Pass|Aviation,EJ/yr,1.535866,1.576815,1.370199,1.343418,1.328005e+00,1.338678e+00,1.326242,...,Aviation,None,None,None,None,None,None,None,None,None
135672,EU27,FE|Transport|Pass|Aviation|Hydrogen,EJ/yr,0.000000,0.000000,0.000000,0.000000,1.968411e-08,4.825360e-08,0.000001,...,Aviation,Hydrogen,None,None,None,None,None,None,None,None
135675,EU27,FE|Transport|Pass|Aviation|Liquids,EJ/yr,1.535866,1.576815,1.370199,1.343418,1.328005e+00,1.338677e+00,1.326241,...,Aviation,Liquids,None,None,None,None,None,None,None,None
135799,EU27,FE|Transport|Pass|Aviation|Liquids||Fossil,EJ/yr,1.520136,1.551127,1.338982,1.267384,1.201112e+00,1.206651e+00,1.183565,...,Aviation,Liquids,,Fossil,None,None,None,None,None,None
135802,EU27,FE|Transport|Pass|Aviation|Liquids||Hydrogen,EJ/yr,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,Aviation,Liquids,,Hydrogen,None,None,None,None,None,None
135805,EU27,FE|Transport|Pass|Aviation|Liquids||Biomass,EJ/yr,0.015730,0.025688,0.031217,0.076034,1.268925e-01,1.320265e-01,0.142675,...,Aviation,Liquids,,Biomass,None,None,None,None,None,None


##### Import JRC files to use as proxy  

The JRC sheets have values for every EU27 countries as well as the total value of EU27.
The objective is to create a new final dataframe that has :
- Transport category (11 total categories)
- Given EU27 value of REMIND for each year
- Share of the choesen JRC year for each country
- Values for each country of EU27 and each year 

In [16]:
remind_to_jrc_category = {
    "FE|Transport|Freight|Road|Heavy": "Heavy goods vehicles",
    "FE|Transport|Freight|Road|Light": "Light commercial vehicles",
    "FE|Transport|Pass|Road|Bus": "Motor coaches, buses and trolley buses",
    "FE|Transport|Pass|Road|LDV|Four Wheelers": "Passenger cars",
    "FE|Transport|Pass|Road|LDV|Two Wheelers": "Powered two-wheelers",
    "FE|Transport|Bunkers|Freight|International Shipping": "Intra-EEA",
    "FE|Transport|Freight|Domestic Shipping": "Domestic navigation",
    "FE|Transport|Pass|Domestic Aviation": "Domestic",
    "FE|Transport|Pass|Aviation": "International - Intra-EEAwUK", 
    "FE|Transport|Pass|Rail": "Rail, metro and tram",
    "FE|Transport|Freight|Rail": "Rail transport"
}

In [17]:
countries = [c for c in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, c)) and c != 'EU27']
categories_JRC = list(remind_to_jrc_category.values())

In [19]:
results = []
for country_code in countries + ['EU27']:
    file_path = os.path.join(base_folder, country_code, f"JRC-IDEES-2021_Transport_{country_code}.xlsx")
    df = pd.read_excel(file_path, sheet_name="Transport", header=0)

    for category in categories_JRC:
        try:
            df.columns = df.columns.map(str)  # Convert all column labels to strings
            df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip()  # Normalize first column

            row = df[df.iloc[:, 0] == category]
            value_2020 = row['2020'].values[0]
            
            results.append({
                'Country': country_code,
                'JRC_Category': category,
                'Value_2020': value_2020
            })
        except IndexError:
            print(f"Missing category {category} in {country_code}")

In [20]:
jrc_df = pd.DataFrame(results)
pivoted = jrc_df.pivot(index='JRC_Category', columns='Country', values='Value_2020')
eu_totals = pivoted['EU27']

# Remove EU27 column for share calc
shares_df = pivoted.drop(columns='EU27').div(eu_totals, axis=0)

# Reshape back to long format
shares_long = shares_df.reset_index().melt(id_vars='JRC_Category', var_name='Country', value_name='Share')
values_long = jrc_df[jrc_df['Country'] != 'EU27']
final_df = pd.merge(values_long, shares_long, on=['Country', 'JRC_Category'])

In [21]:
final_df

,Country,JRC_Category,Value_2020,Share
0,AT,Heavy goods vehicles,46987.000000,0.026916
1,AT,Light commercial vehicles,432.167201,0.004951
2,AT,"Motor coaches, buses and trolley buses",7444.142947,0.025622
3,AT,Passenger cars,65176.000000,0.018532
4,AT,Powered two-wheelers,1537.282688,0.015291
...,...,...,...,...
292,SK,Domestic navigation,834.000000,0.003995
293,SK,Domestic,0.000000,0.000000
294,SK,International - Intra-EEAwUK,254.260995,0.001563
295,SK,"Rail, metro and tram",2301.386401,0.008253


In [22]:
category_share_map = {}
for remind_cat, jrc_cat in remind_to_jrc_category.items():
    df = final_df[final_df["JRC_Category"] == jrc_cat].copy()
    df["REMIND_Category"] = remind_cat
    category_share_map[remind_cat] = df[["REMIND_Category", "Country", "Value_2020", "Share"]]

In [23]:
category_share_map

{'FE|Transport|Freight|Road|Heavy':                      REMIND_Category Country     Value_2020     Share
 0    FE|Transport|Freight|Road|Heavy      AT   46987.000000  0.026916
 11   FE|Transport|Freight|Road|Heavy      BE   51947.000000  0.029758
 22   FE|Transport|Freight|Road|Heavy      BG   11005.000000  0.006304
 33   FE|Transport|Freight|Road|Heavy      CY     672.000000  0.000385
 44   FE|Transport|Freight|Road|Heavy      CZ   51710.000000  0.029622
 55   FE|Transport|Freight|Road|Heavy      DE  466414.000000  0.267185
 66   FE|Transport|Freight|Road|Heavy      DK   20209.000000  0.011577
 77   FE|Transport|Freight|Road|Heavy      EE    2751.000000  0.001576
 88   FE|Transport|Freight|Road|Heavy      EL   16783.000000  0.009614
 99   FE|Transport|Freight|Road|Heavy      ES  207453.000000  0.118839
 110  FE|Transport|Freight|Road|Heavy      FI   28942.000000  0.016579
 121  FE|Transport|Freight|Road|Heavy      FR  280099.000000  0.160454
 132  FE|Transport|Freight|Road|Heavy     

In [24]:
final_df = pd.concat(category_share_map.values(), ignore_index=True)
final_df

,REMIND_Category,Country,Value_2020,Share
0,FE|Transport|Freight|Road|Heavy,AT,46987.0,0.026916
1,FE|Transport|Freight|Road|Heavy,BE,51947.0,0.029758
2,FE|Transport|Freight|Road|Heavy,BG,11005.0,0.006304
3,FE|Transport|Freight|Road|Heavy,CY,672.0,0.000385
4,FE|Transport|Freight|Road|Heavy,CZ,51710.0,0.029622
...,...,...,...,...
292,FE|Transport|Freight|Rail,PT,2302.0,0.006101
293,FE|Transport|Freight|Rail,RO,12291.0,0.032576
294,FE|Transport|Freight|Rail,SE,22094.0,0.058557
295,FE|Transport|Freight|Rail,SI,4726.0,0.012526


In [25]:
all_distributed = []

for category in categories_REMIND:
    remind_df = filtered_data[category]

    # Select only EU27 rows
    eu27_df = remind_df[remind_df["Region"] == "EU27"]
    if eu27_df.empty:
        print(f"No EU27 data for {category}")
        continue

    # Loop through each subcategory (row), preserving fuel/vehicle distinction
    for _, eu_row in eu27_df.iterrows():
        variable = eu_row["Variable"]
        
        if variable == category:
            subcategory_label = category  # no suffix
        else:
            subcategory_label = f"{category}|{variable.replace(category + '|', '')}"

        # Extract year columns
        year_cols = [col for col in eu_row.index if col.isdigit()]
        remind_values = eu_row[year_cols].astype(float)

        # Use JRC shares (based on main REMIND category)
        shares_df = final_df[final_df["REMIND_Category"] == category][["Country", "Share"]].copy()

        for year in year_cols:
            eu_value = remind_values[year]
            if pd.isna(eu_value):
                continue

            shares_df["Value"] = shares_df["Share"] * eu_value
            shares_df["Year"] = int(year)
            shares_df["Category"] = subcategory_label

            # Append distributed values
            all_distributed.append(shares_df[["Category", "Country", "Year", "Value"]])

            # Add EU27 total explicitly
            all_distributed.append(pd.DataFrame([{
                "Category": subcategory_label,
                "Country": "EU27",
                "Year": int(year),
                "Value": eu_value
            }]))


No EU27 data for FE|Bunkers|Pass|International Aviation


In [26]:
all_distributed

[                                             Category Country  Year  Value
 0   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      AT  2005    0.0
 1   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      BE  2005    0.0
 2   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      BG  2005    0.0
 3   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      CY  2005    0.0
 4   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      CZ  2005    0.0
 5   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      DE  2005    0.0
 6   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      DK  2005    0.0
 7   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      EE  2005    0.0
 8   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      EL  2005    0.0
 9   FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      ES  2005    0.0
 10  FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      FI  2005    0.0
 11  FE|Transport|Freight|Road|Heavy|Truck(18t)|Gas...      FR  2005    0.0
 12  FE|Tran

In [27]:
distributed_df = pd.concat(all_distributed, ignore_index=True)

In [28]:
top_level_df = distributed_df[distributed_df["Category"].isin(categories_REMIND)]
top_level_df

,Category,Country,Year,Value
9240,FE|Transport|Freight|Road|Heavy,AT,2005,0.053872
9241,FE|Transport|Freight|Road|Heavy,BE,2005,0.059559
9242,FE|Transport|Freight|Road|Heavy,BG,2005,0.012618
9243,FE|Transport|Freight|Road|Heavy,CY,2005,0.000770
9244,FE|Transport|Freight|Road|Heavy,CZ,2005,0.059287
...,...,...,...,...
73915,FE|Transport|Freight|Rail,RO,2050,0.003872
73916,FE|Transport|Freight|Rail,SE,2050,0.006960
73917,FE|Transport|Freight|Rail,SI,2050,0.001489
73918,FE|Transport|Freight|Rail,SK,2050,0.002176


In [29]:
check = distributed_df[
    (distributed_df["Category"] == "FE|Transport|Pass|Aviation") &
    (distributed_df["Country"] == "EU27")
]
print(check)


                         Category Country  Year     Value
66667  FE|Transport|Pass|Aviation    EU27  2005  1.535866
66695  FE|Transport|Pass|Aviation    EU27  2010  1.576815
66723  FE|Transport|Pass|Aviation    EU27  2015  1.370199
66751  FE|Transport|Pass|Aviation    EU27  2020  1.343418
66779  FE|Transport|Pass|Aviation    EU27  2025  1.328005
66807  FE|Transport|Pass|Aviation    EU27  2030  1.338678
66835  FE|Transport|Pass|Aviation    EU27  2035  1.326242
66863  FE|Transport|Pass|Aviation    EU27  2040  1.298374
66891  FE|Transport|Pass|Aviation    EU27  2045  1.268770
66919  FE|Transport|Pass|Aviation    EU27  2050  1.242219


In [30]:
filtered_data['FE|Transport|Pass|Aviation']

,Region,Variable,Unit,2005,2010,2015,2020,2025,2030,2035,...,Variable_3,Variable_4,Variable_5,Variable_6,Variable_7,Variable_8,Variable_9,Variable_10,Variable_11,Variable_12
135300,EU27,FE|Transport|Pass|Aviation,EJ/yr,1.535866,1.576815,1.370199,1.343418,1.328005e+00,1.338678e+00,1.326242,...,Aviation,None,None,None,None,None,None,None,None,None
135672,EU27,FE|Transport|Pass|Aviation|Hydrogen,EJ/yr,0.000000,0.000000,0.000000,0.000000,1.968411e-08,4.825360e-08,0.000001,...,Aviation,Hydrogen,None,None,None,None,None,None,None,None
135675,EU27,FE|Transport|Pass|Aviation|Liquids,EJ/yr,1.535866,1.576815,1.370199,1.343418,1.328005e+00,1.338677e+00,1.326241,...,Aviation,Liquids,None,None,None,None,None,None,None,None
135799,EU27,FE|Transport|Pass|Aviation|Liquids||Fossil,EJ/yr,1.520136,1.551127,1.338982,1.267384,1.201112e+00,1.206651e+00,1.183565,...,Aviation,Liquids,,Fossil,None,None,None,None,None,None
135802,EU27,FE|Transport|Pass|Aviation|Liquids||Hydrogen,EJ/yr,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,...,Aviation,Liquids,,Hydrogen,None,None,None,None,None,None
135805,EU27,FE|Transport|Pass|Aviation|Liquids||Biomass,EJ/yr,0.015730,0.025688,0.031217,0.076034,1.268925e-01,1.320265e-01,0.142675,...,Aviation,Liquids,,Biomass,None,None,None,None,None,None


In [ ]:
if save_output_file == True:
    distributed_df.to_csv(output_path, index=False)